My friend VictORia and I are planning a road trip.

We want to drive from Madrid to Copenhagen and we've seen there are a lot of ways to do it.

Most probably, even if we wanted to go through Budapest, we cannot go that path. We have a limited budget for fuel of 73€, and we want to get the shortest possible way.

Can you help me solve this problem?

## Solution modeled with binary variables

**Goal**

Shortest route given a budget for fuel

$$\min \sum_{u,v \in P} d_{uv} x_{uv}$$

**Decisions**:

$$x_{uv} \in \{0,1\} \text{ to use road } (u,v) \text{connecting city u and v}$$


**Data:**

Set of cities $C$

Roads from city to city (network) $R=\{(u,v)|u,v \in C\}$

Distance from city to city $d_{uv}$

Cost of traversing a road $c_{uv}$

Start node $\{s\}$

Target node $\{t\}$

Budget $B$ 



**Constraints**

| Description    | Expression |
| -------- | ------- |
| Choose exactly one road that flows out at start node $s$  | $\sum_v x_{sv} = 1$    |
| Choose exactly one road that flows in at target node $t$ | $\sum_u x_{ut} = 1$ |
| Don’t end at initial node | $\forall u,s \in R: x_{us} = 0$ |
| Don’t start at target node | $\forall t,v \in R: x_{tv} = 0$ |
| The net flow of the rest of the nodes must be zero | $\forall w \in C - \{s, t\}: \sum_u x_{uw} - \sum_v x_{wv} = 0 $ |
| Subtour elimination ? | |
| Stay within the budget| $\sum_{u,v} c_{uv} x_{uv} \leq B$ |

In [1]:
def get_instance(input_filename):
    with open(input_filename, "r") as file:
        data = file.read()

    # Split the data into lines
    lines = data.strip().split('\n')
    l = 0

    # Skip comment lines
    while lines[l].startswith('#') or lines[l] == '':
        l += 1
    
    # get metadata parameters (always first line)
    metadata = map(int, lines[l].split())
    data = []
    l += 1

    # Process each line
    while l < len(lines):
        data.append(list(map(int, lines[l].split())))
        l += 1

    return *metadata, data

def parse_to_zero_index(data, indexes):
    for i in range(len(data)):
        for idx in indexes:
            data[i][idx] -= 1
    return

In [2]:
num_cities, num_roads, max_budget, roads = get_instance("data/euro_trip.txt")
s, t = 0, num_cities - 1

parse_to_zero_index(roads, (0, 1))
#num_rooms = num_events # worst-case scenario
# num_rooms = 15
# Print the extracted data
print(f"Number of cities: {num_cities}")
print(f"Number of roads: {num_roads}")
print(f"Max budget: {max_budget}")
print(f"Madrid at {s}, Copenhagen at {t}")
# print("Conflicts:")
# for conflict in conflicts:
#     print(conflict)

Number of cities: 100
Number of roads: 955
Max budget: 73
Madrid at 0, Copenhagen at 99


In [3]:
from ortools.sat.python import cp_model

model = cp_model.CpModel()
solver = cp_model.CpSolver()

In [4]:
x = [[None] * num_cities for _ in range(num_cities)]
c = [[None] * num_cities for _ in range(num_cities)]
d = [[None] * num_cities for _ in range(num_cities)]

for u, v, dist, cost in roads:
    x[u][v] = model.NewBoolVar(name=f'path_{u}_{v}_chosen')
    d[u][v] = dist
    c[u][v] = cost

In [5]:
model.Add(sum(x[s][v] for v in range(num_cities) if x[s][v] is not None) == 1)

In [6]:
model.Add(sum(x[u][t] for u in range(num_cities) if x[u][t] is not None) == 1)

In [7]:
for u in range(num_cities):
    if x[u][s] is not None:
        print(x[u][s])
        model.Add(x[u][s] == 0)

path_10_0_chosen
path_23_0_chosen
path_78_0_chosen
path_79_0_chosen


In [8]:
for v in range(num_cities):
    if x[t][v] is not None:
        print(x[t][v])
        model.Add(x[t][v] == 0)

path_99_2_chosen
path_99_8_chosen
path_99_60_chosen
path_99_63_chosen
path_99_76_chosen
path_99_78_chosen
path_99_82_chosen
path_99_87_chosen
path_99_90_chosen
path_99_98_chosen


In [9]:
for w in range(num_cities):
    if w in (s, t):
        continue
    model.Add(
        sum(x[w][v] for v in range(num_cities) if x[w][v] is not None) - \
        sum(x[u][w] for u in range(num_cities) if x[u][w] is not None) == 0
    )

In [10]:
model.Add(
    sum(c[u][v] * x[u][v] 
        for u in range(num_cities) 
        for v in range(num_cities) 
        if x[u][v] is not None
        ) <= max_budget
    )

In [11]:
# Specify the type of problem. In this case, we want to minimize the objective function
solver.parameters.num_search_workers = 8
solver.parameters.max_time_in_seconds = 120
model.Minimize(
    sum(d[u][v] * x[u][v] 
        for u in range(num_cities) 
        for v in range(num_cities) 
        if x[u][v] is not None
    )
)

In [13]:
# Call the solver method to find the optimal solution
callback = cp_model.ObjectiveSolutionPrinter()
or_status = solver.SolveWithSolutionCallback(model, callback)
status = solver.StatusName(or_status)

if status in ["OPTIMAL", "FEASIBLE"]:
    print(f'Solution: Total distance traveled = {solver.ObjectiveValue()}')
else:
    print('A solution could not be found, check the problem specification')

Solution 0, time = 0.07 s, objective = 311
Solution 1, time = 0.08 s, objective = 272
Solution 2, time = 0.08 s, objective = 247
Solution 3, time = 0.08 s, objective = 189
Solution 4, time = 0.08 s, objective = 175
Solution 5, time = 0.09 s, objective = 167
Solution 6, time = 0.09 s, objective = 131
Solution: Total distance traveled = 131.0


In [19]:
solution = [[None] * num_cities for _ in range(num_cities)]
for u in range(num_cities):
    for v in range(num_cities):
        if x[u][v] is not None and solver.Value(x[u][v]) == 1:
            print((u,v), solver.Value(x[u][v]))
# [solver.Value(x[s][v]) for v in range(num_cities) if x[s][v] is not None]

(0, 36) 1
(1, 99) 1
(36, 40) 1
(40, 1) 1
